# 5-5-如何访问文件的状态

## 实际案例
某些项目中，我们需要获得文件状态，例如：
1. 文件类型(普通文件、目录、符号链接、设备文件)
2. 文件的访问权限
3. 文件的最后访问、修改、节点状态更改(修改文件属性)时间
4. 普通文件大小

## 系统调用 方式

In [1]:
import os

In [2]:
fd = os.open("./5-5-test/q.py", os.O_RDONLY) # 文件描述符对象,其实就是个数字，可以给系统调用的其他方法用
os.read(fd,10)

b'import os\n'

In [3]:
a = os.stat("./5-5-test/q.py")
a

os.stat_result(st_mode=33277, st_ino=2604260, st_dev=64514, st_nlink=1, st_uid=1000, st_gid=1000, st_size=32, st_atime=1636764126, st_mtime=1636764076, st_ctime=1636764076)

In [4]:
print(bin(a.st_mode)) #后9位表示User,Group,Other的权限，前面的都是表示文件类型
!ls -l ./5-5-test/q.py

0b1000000111111101
-rwxrwxr-x 1 ubuntu ubuntu 32 Nov 13 08:41 ./5-5-test/q.py


`st_proc`表示当前文件所在设备,比如`/proc`的`st_proc`就和一般文件不同，因为它是在内存中的，一般文件是在磁盘中的。  
`st_nlink`表示文件的链接数(硬链接)  
`st_uid`指文件的所有者的id  
`st_gid`指组id  
`st_size`指文件数据大小，12个字节  
`st_atime`访问时间  
`st_mtime`修改时间  
`st_ctime`节点状态修改时间

### stat 模块
注意下面只是演示，还有很多常量和参数，要用去看文档

In [5]:
import stat

In [6]:
s = os.stat("./5-5-test/q.py")
s2 = os.stat("./5-5-test/d")
s3 = os.stat("./5-5-test/1.txt")

#### 文件类型

In [7]:
print(stat.S_IFDIR & s.st_mode)
print(stat.S_IFCHR & s.st_mode)
print(stat.S_IFREG & s.st_mode)
print("-"*10)
print(stat.S_ISDIR(s2.st_mode))

0
0
32768
----------
True


In [8]:
print(stat.S_ISLNK(s3.st_mode)) # s3 是符号链接，然而却显示不是，因为os.stat发现它是符号链接，就跟随它了
s3 = os.stat("5-5-test/1.txt",follow_symlinks=False) # 或者 os.lstat() 函数
print(stat.S_ISLNK(s3.st_mode))

False
True


#### 文件权限

In [9]:
print(stat.S_IRUSR & s.st_mode) # 判断用户是否可读
print(stat.S_IXOTH & s.st_mode) # 其他人是否可执行

256
1


#### 时间

In [10]:
import time

In [11]:
time.localtime(s.st_atime)

time.struct_time(tm_year=2021, tm_mon=11, tm_mday=13, tm_hour=8, tm_min=42, tm_sec=6, tm_wday=5, tm_yday=317, tm_isdst=0)

## 快捷函数 os.path下函数

In [12]:
import os.path as op

### 文件类型

In [13]:
op.isdir("./5-5-test/d")

True

### 文件权限 （不能）

### 时间

In [14]:
op.getatime("./5-5-test/q.py")

1636764126.3916626

### 获取文件大小

In [15]:
op.getsize("./5-5-test/q.py")

32